In [21]:
def ideal_critical_d(df):
    """Считает идеальный критический диаметр, возвращает датафрейм с добавленным признаком"""
    for x in df['C']:
        if x <= 0.39:
            coef1 = 0.54*x
        else:
            coef1 = 0.171+0.001*x+0.265*x*x
    for x in df['Mn']:
        if x <= 1.2:
            coef2 = 3.3333*x+1
        else:
            coef2 = 5.1*x-1.12
    result = coef1*coef2*(1+0.7*df['Si'])*(1+0.363*df['Ni'])*(1+2.16*df['Cr'])*(
            1+3.0*df['Mo'])*(1+0.365*df['Cu'])*(1+1.73*df['V'])
    df['ICD'] = result
    return df

In [22]:
def calc_C_coef(df):
    """Считает углеродный коэффициент, возвращает датафрейм с добавленным признаком"""
    df['C-coef'] = ((df['C'] + df['Mn'])/6.0 + (df['Cr'] + df['Mo'] + df['V'])/5.0 +
                        (df['Ni'] + df['Cu'])/15.0)
    return df

In [23]:
def calc_quenching_param(df):
    """Считает параметр закалки, возвращает датафрейм с добавленным признаком"""
    df['Параметр закалка'] = 1/(1/(df[['2 зона по ВТР закалка','3 зона по ВТР закалка']].mean(axis=1) 
                            + 273) - (2.303*1.986*np.log10(61*df['шаг балок закалочная печь, сек']/3600)/110000))-273
    return df

In [24]:
def calc_tempering_param(df):
    """Считает параметр отпуска, возвращает датафрейм с добавленным признаком"""
    df['Параметр отпуск'] = (df[['2 зона ВТР и уставка отпуск','4 зона ВТР и уставка отпуск','5 зона ВТР и уставка отпуск']].mean(axis=1)
                        + 273)*(20+np.log(94*df['шаг балок отпускная печь, сек']/3600))*1e-3
    return df

In [25]:
def calc_all_features(df):
    """Считает и добавляет все необходимые признаки в датафрейм"""
    df = ideal_critical_d(df)
    df = calc_C_coef(df)
    df = calc_quenching_param(df)
    df = calc_tempering_param(df)
    return df

In [26]:
def lowerstr(f):
    """Приводим все символы в номерах партий к нижнему регистру"""
    for col in f:
        if '№ партии' in col:
            f['№ партии'] = f['№ партии'].apply(lambda x: str(x))            
            f['№ партии'] = f['№ партии'].str.lower()
        if '№ плавки' in col:
            f['№ плавки'] = f['№ плавки'].str.lower()
    return f

In [27]:
def make_prepared(df, exp_df, chem):
    """Подготавливает датафрейм"""
#     обединяем испытания с режимами и химией
    df_merge = pd.merge(exp_df, df, how ='left', on = ['№ плавки','№ партии'])
#     заполняем пустую химию средней (или нулями)
    df_merge = df_merge.fillna(value = chem)
#     удаляем строки без режимов
    df_merge = df_merge[~pd.isnull(df_merge['C'])]
#     добавляем рассчитаные признаки
    df_merge = calc_all_features(df_merge)
    df_merge = df_merge[df_merge['Cr'] < 10]
    df_merge.reset_index(drop=True, inplace=True)
    print(df_merge.shape)
    return df_merge

In [28]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
# warnings.filterwarnings("ignore")
import sklearn
import pandas as pd
import numpy as np
import os

pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

In [29]:
df_modes = pd.read_excel('merge_table_mean_chem.xlsx')
df_valid = pd.read_excel('valid_table_mean_chem.xlsx')
table_stretching = pd.read_excel('table_stretching_13april.xlsx')
table_stretching = lowerstr(table_stretching)
df_valid['№ партии'] = df_valid['№ партии'].apply(lambda x: str(x).replace('.0', ''))
print(table_stretching.shape)

(16185, 5)


In [30]:
ls_all = [
    'Поставщик',
    'НД (гост, ту)',
    'ГОСТ, ТУ',
    'Дата форм. цехом',
    'марка стали',
    '№ плавки',
    'кол-во, шт.',
    '№ партии',
    '1 зона по ВТР закалка', 
    '1 зона уставка закалка',
    '1 зона фактическая температура закалка', 
    '2 зона по ВТР закалка',
    '3 зона по ВТР закалка', 
    '3 зона уставка закалка',
    '3 зона фактическая температура закалка',
    'Температура трубы на выгрузке из печи закалки',
    'Скорость прохождения трубы через спрейер, м/с',
    't˚ C трубы после спреера', 
    'Расход воды на закалочный спрейер, м3',
    'время выдержки в закалочной ванне, сек.',
    't˚ C воды в закалочной ванне', 
    't˚ C трубы после ванны',
    '1 зона ВТР и уставка отпуск', 
    '1 зона фактическая температура отпуск',
    '2 зона ВТР и уставка отпуск', 
    '3 зона ВТР и уставка отпуск',
    '3 зона фактическая температура отпуск', 
    '4 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск', 
    '5 зона фактическая температура отпуск',
    'Температура трубы на выгрузке из печи отпуска',
    'шаг балок закалочная печь, сек', 
    'шаг балок отпускная печь, сек',
    'диаметр', 
    'толщина стенки', 
    'C', 
    'Mn', 
    'Si', 
    'P', 
    'S',
    'Cr', 
    'Ni', 
    'Cu', 
    'Al', 
    'V', 
    'Ti', 
    'Nb', 
    'Mo', 
    'N', 
    'Параметр',
    'C-coef',
    'ICD'
]

chem = {#'Mn': 0, 'Si': 0, 
#         'P': 0, 
#         'Cr': 0, 'Ni': 0, 'Cu': 0, 'Al': 0, 
    'V': 0, 
    'Mo': 0, 
        'Ti': 0, 'Nb': 0, 'N': 0.004, 'S': 0}


In [31]:
df_merge = make_prepared(df_modes, table_stretching, chem)
df_merge_valid = make_prepared(df_valid, table_stretching, chem)

(6517, 54)
(228, 43)


In [32]:
df_merge.to_excel('prepared_hard.xlsx')
df_merge_valid.to_excel('prepared_hard_valid.xlsx')